In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy as sp
import xgboost as xgb
from scipy import sparse
from IPython.display import Image
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
sns.set(style="ticks", color_codes=True)
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
import os
import gensim
import json
from sklearn.metrics import mean_squared_error
from datetime import datetime
import dateutil
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

random_state=2021

In [2]:
!pip install -U textblob
# from textblob import TextBlob

     |████████████████████████████████| 636 kB 6.9 MB/s 
  Attempting uninstall: textblob
    Found existing installation: textblob 0.15.3
    Uninstalling textblob-0.15.3:
      Successfully uninstalled textblob-0.15.3


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# dataset = read_all()
df=pd.read_csv('/content/drive/MyDrive/NLP/final_dataset.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6,9,14,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
dropped = ['following','posts_count','datetime', 'comments', 'location','image_url','external_url','media_type','business_email','is_verified','profile_image_link','id','url','thumbnails']
data = df.drop(dropped,axis=1)
data

,account,caption,profile_name,biography,likes,followers
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,🌎🧳🔥🥇🎨🍷☕\n 🇻🇪 🇨🇴\nEMPRESARIO SOÑADOR\...,NaN,1244.0
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Consultant Plastic and Reconstructive Surgeon,NaN,13.0
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
74544,thesavoylondon,"Alongside hospitality, the performing arts hav...",NaN,At the forefront of the luxury hotel scene for...,4079,167394.0
74545,thesavoylondon,This! Isn’t this what we are looking forward ...,NaN,At the forefront of the luxury hotel scene for...,1621,167394.0
74546,thesavoylondon,"For those of you who are missing London town, ...",NaN,At the forefront of the luxury hotel scene for...,1548,167394.0
74547,thesavoylondon,While we’re closed we’re going to try and keep...,NaN,At the forefront of the luxury hotel scene for...,2637,167394.0


In [8]:
#drop the rows which contain at least NaN values
data.dropna(axis=0,  how='any',inplace=True)

In [9]:
from textblob import TextBlob


def sentiment_analysis(caption):
 def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity
  
 #Create a function to get the polarity
 def getPolarity(caption):
   return TextBlob(caption).sentiment.polarity
  
 #Create two new columns ‘Subjectivity’ & ‘Polarity’
 data['TextBlob_Subjectivity'] =    data['caption'].apply(getSubjectivity)
 data ['TextBlob_Polarity'] = data['caption'].apply(getPolarity)


 def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'


 data ['TextBlob_Analysis'] = data['TextBlob_Polarity'].apply(getAnalysis )
 return data

In [10]:
sentiment_analysis(data['caption'])

,account,caption,profile_name,biography,likes,followers,TextBlob_Subjectivity,TextBlob_Polarity,TextBlob_Analysis
10,chelsea_bridge_clinic,𝗗𝗶𝗱 𝘆𝗼𝘂 𝗸𝗻𝗼𝘄 𝘁𝗵𝗮𝘁 𝘆𝗼𝘂 𝗰𝗮𝗻 𝘁𝗲𝘀𝘁 𝗳𝗼𝗿 𝗖𝗼𝘃𝗶𝗱 𝘄𝗶𝘁𝗵 ...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,14.0,1093.0,0.632390,0.282576,Positive
11,chelsea_bridge_clinic,𝗧𝗛𝗔𝗡𝗞 𝗬𝗢𝗨 👏\n.\nWe are very grateful for all t...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,12.0,1093.0,0.405556,0.168056,Positive
12,numberoneedin,"Hand dived Isle of Jura scallop, smoked roe, l...",Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,100.0,4202.0,0.000000,0.000000,Neutral
13,numberoneedin,Jerusalem Artichoke prepared as your first des...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,81.0,4202.0,0.333333,0.250000,Positive
14,numberoneedin,Main course from our seasonal 7 course menu: B...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,146.0,4202.0,0.333333,0.166667,Positive
...,...,...,...,...,...,...,...,...,...
74513,seacontainersldn,Scenario: You and your loved one are in desper...,Sea Containers London,Explore the vibrant South Bank from your Londo...,438,46265.0,0.680952,0.314286,Positive
74514,seacontainersldn,We’ve launched our ‘Nominate you Hero’ Vol. II...,Sea Containers London,Explore the vibrant South Bank from your Londo...,221,46265.0,0.658673,0.251531,Positive
74515,seacontainersldn,It’s not often that the lobby is shot at this ...,Sea Containers London,Explore the vibrant South Bank from your Londo...,859,46265.0,0.950000,0.600000,Positive
74516,seacontainersldn,"We are supporting @makeitblueuk, a community i...",Sea Containers London,Explore the vibrant South Bank from your Londo...,529,46265.0,0.439583,0.106250,Positive


In [11]:
dropped = ['TextBlob_Polarity','TextBlob_Subjectivity']
data = data.drop(dropped,axis=1)
data

,account,caption,profile_name,biography,likes,followers,TextBlob_Analysis
10,chelsea_bridge_clinic,𝗗𝗶𝗱 𝘆𝗼𝘂 𝗸𝗻𝗼𝘄 𝘁𝗵𝗮𝘁 𝘆𝗼𝘂 𝗰𝗮𝗻 𝘁𝗲𝘀𝘁 𝗳𝗼𝗿 𝗖𝗼𝘃𝗶𝗱 𝘄𝗶𝘁𝗵 ...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,14.0,1093.0,Positive
11,chelsea_bridge_clinic,𝗧𝗛𝗔𝗡𝗞 𝗬𝗢𝗨 👏\n.\nWe are very grateful for all t...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,12.0,1093.0,Positive
12,numberoneedin,"Hand dived Isle of Jura scallop, smoked roe, l...",Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,100.0,4202.0,Neutral
13,numberoneedin,Jerusalem Artichoke prepared as your first des...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,81.0,4202.0,Positive
14,numberoneedin,Main course from our seasonal 7 course menu: B...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,146.0,4202.0,Positive
...,...,...,...,...,...,...,...
74513,seacontainersldn,Scenario: You and your loved one are in desper...,Sea Containers London,Explore the vibrant South Bank from your Londo...,438,46265.0,Positive
74514,seacontainersldn,We’ve launched our ‘Nominate you Hero’ Vol. II...,Sea Containers London,Explore the vibrant South Bank from your Londo...,221,46265.0,Positive
74515,seacontainersldn,It’s not often that the lobby is shot at this ...,Sea Containers London,Explore the vibrant South Bank from your Londo...,859,46265.0,Positive
74516,seacontainersldn,"We are supporting @makeitblueuk, a community i...",Sea Containers London,Explore the vibrant South Bank from your Londo...,529,46265.0,Positive


In [12]:
data = data.iloc[: , 1:]

In [13]:
data

,caption,profile_name,biography,likes,followers,TextBlob_Analysis
10,𝗗𝗶𝗱 𝘆𝗼𝘂 𝗸𝗻𝗼𝘄 𝘁𝗵𝗮𝘁 𝘆𝗼𝘂 𝗰𝗮𝗻 𝘁𝗲𝘀𝘁 𝗳𝗼𝗿 𝗖𝗼𝘃𝗶𝗱 𝘄𝗶𝘁𝗵 ...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,14.0,1093.0,Positive
11,𝗧𝗛𝗔𝗡𝗞 𝗬𝗢𝗨 👏\n.\nWe are very grateful for all t...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,12.0,1093.0,Positive
12,"Hand dived Isle of Jura scallop, smoked roe, l...",Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,100.0,4202.0,Neutral
13,Jerusalem Artichoke prepared as your first des...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,81.0,4202.0,Positive
14,Main course from our seasonal 7 course menu: B...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,146.0,4202.0,Positive
...,...,...,...,...,...,...
74513,Scenario: You and your loved one are in desper...,Sea Containers London,Explore the vibrant South Bank from your Londo...,438,46265.0,Positive
74514,We’ve launched our ‘Nominate you Hero’ Vol. II...,Sea Containers London,Explore the vibrant South Bank from your Londo...,221,46265.0,Positive
74515,It’s not often that the lobby is shot at this ...,Sea Containers London,Explore the vibrant South Bank from your Londo...,859,46265.0,Positive
74516,"We are supporting @makeitblueuk, a community i...",Sea Containers London,Explore the vibrant South Bank from your Londo...,529,46265.0,Positive


In [14]:
!pip install advertools

     |████████████████████████████████| 287 kB 7.9 MB/s 
     |████████████████████████████████| 264 kB 62.3 MB/s 
     |████████████████████████████████| 3.1 MB 56.4 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 87 kB 7.3 MB/s 
     |████████████████████████████████| 47 kB 4.9 MB/s 
     |████████████████████████████████| 251 kB 54.7 MB/s 
     |████████████████████████████████| 3.6 MB 48.9 MB/s 
     |████████████████████████████████| 74 kB 3.4 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11516 sha256=f9e18a97a6cb6a5797d4a9e8f0b61b85a1072eff4793fcd7e88b54475f61e999
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa69a339198e08bb63b7da2c45933a3428b29ec454
Successfully built PyDispatcher


In [15]:
import advertools as adv

hashtag_summary = adv.extract_hashtags(data['caption'])
hashtag_summary.keys()

dict_keys(['hashtags', 'hashtags_flat', 'hashtag_counts', 'hashtag_freq', 'top_hashtags', 'overview'])

In [16]:
hashtag_summary['hashtags'][:10]

[['#londontesting',
  '#chelseabridgeclinic',
  '#london',
  '#test',
  '#antigentest',
  '#lateralflowtest',
  '#testforcovid',
  '#covidtesting'],
 ['#thankyou',
  '#thankful',
  '#feedback',
  '#chlseabridgeclinic',
  '#london',
  '#covidtesting',
  '#covidtest',
  '#support',
  '#testfortravels'],
 ['#edinburghfoodies',
  '#edinburghrestaurants',
  '#numberone',
  '#thebalmoral',
  '#roccofortehotels',
  '#roccofortefriends',
  '#edinburgh',
  '#igersedinburgh',
  '#scotland',
  '#visitedinburgh',
  '#foreveredinburgh',
  '#visitscotland',
  '#scotlandisnow',
  '#travel',
  '#traveller',
  '#balmoralmoments',
  '#edinburghsnapshots',
  '#edinburghcity',
  '#edinburghlife',
  '#hotellife',
  '#photography',
  '#picoftheday',
  '#photooftheday',
  '#igdaily',
  '#instadaily'],
 ['#michelin',
  '#michelinstar',
  '#michelinuk',
  '#edinburghfoodies',
  '#edinburghrestaurants',
  '#numberone',
  '#thebalmoral',
  '#roccofortehotels',
  '#roccofortefriends',
  '#edinburgh',
  '#igersedi

In [17]:
# dropped = ['hashtags']
# data = data.drop(dropped,axis=1)
# data

##emoji

In [18]:
# hashtag_words = adv
# hashtag_words.keys()

In [19]:
emoji_summary = adv.extract_emoji(data['caption'])
emoji_summary.keys()

dict_keys(['emoji', 'emoji_text', 'emoji_flat', 'emoji_flat_text', 'emoji_counts', 'emoji_freq', 'top_emoji', 'top_emoji_text', 'top_emoji_groups', 'top_emoji_sub_groups', 'overview'])

In [20]:
emoji_summary['emoji_flat'][:10]

['⏱',
 '📄',
 '🧑🏻\u200d⚕️',
 '👨\u200d👩\u200d👧\u200d👦',
 '👫',
 '🧑\u200d💻',
 '\U0001f90d',
 '📌',
 '🦠',
 '🤝']

In [21]:
emoji_summary

{'emoji': [['⏱',
   '📄',
   '🧑🏻\u200d⚕️',
   '👨\u200d👩\u200d👧\u200d👦',
   '👫',
   '🧑\u200d💻',
   '\U0001f90d',
   '📌',
   '🦠',
   '🤝'],
  ['👏', '🧡'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  ['📸'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  ['😂'],
  [],
  [],
  ['💋', '👄', '💄'],
  [],
  [],
  [],
  ['🥰'],
  ['🥰'],
  ['🙏', '❤️'],
  ['👩\u200d❤️\u200d👨'],
  ['🙏', '❤️'],
  ['💋', '💋'],
  ['💔'],
  ['✨', '✨', '🎶', '🎸', '💃🏻', '🕺🏻'],
  ['🐶', '🍻'],
  ['\U0001f90d', '💃🏻', '🍷', '🍻'],
  ['🏉', '💃🏻', '🎶', '📲', '✨'],
  ['🎃', '🕺🏻', '👻', '🍻', '✨'],
  ['📲', '📲', '🍻', '🗣'],
  ['🥃', '✨'],
  ['✨', 

###mention

In [22]:
mention_summary = adv.extract_mentions(data['caption'])
mention_summary.keys()

dict_keys(['mentions', 'mentions_flat', 'mention_counts', 'mention_freq', 'top_mentions', 'overview'])

In [23]:
mention_summary['mentions'][:15]

[[],
 [],
 [],
 [],
 ['@castlegamescotland', '@wellocksfood'],
 [],
 ['@_mark_donald',
  '@theglenturretlalique',
  '@cailbruich',
  '@unalomebygc',
  '@unalomebygc',
  '@martin_wishart',
  '@thekitchin',
  '@condita_restaurant',
  '@thepeatinn',
  '@restaurant_andrew_fairlie',
  '@lochbayskye'],
 ['@liberty_wines', '@wsetglobal'],
 [],
 [],
 ['@chateaulepuy'],
 [],
 [],
 ['@hardensbites'],
 []]

##Question

In [24]:
question_summary = adv.extract_questions(data['caption'])

In [25]:
question_summary['overview']

{'num_posts': 73232,
 'num_question_marks': 17602,
 'question_marks_per_post': 0.24035940572427353,
 'unique_question_marks': 5}

##Intense words

In [26]:
intense_summary = adv.extract_intense_words(data['caption'], min_reps=3)

In [27]:
intense_summary['top_intense_words'][:20]

[('⠀⠀⠀⠀⠀⠀⠀⠀⠀', 542),
 ('...', 488),
 ('.⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀', 428),
 ('#eeeeeats', 319),
 ('www.dermamedical.co.uk', 240),
 ('\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063\u2063', 236),
 ('www.dermamedical.net.au', 204),
 ('www.dermamedical.se', 200),
 ('www.lmaclinic.com', 191),
 ('www.dermamedical.ae', 188),
 ('#coffeeedinburgh', 173),
 ('#111SKIN', 166),
 ('———————————————————————————', 155),
 ('www.drmaryamzamani.com', 153),
 ('・・・', 145),
 ('www.perfecteyesltd.com', 144),
 ('.⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀', 142),
 ('\u2063\u2063\u2063', 141),
 ('www.shumailas.click', 137),
 ('patients.⠀⠀⠀⠀⠀⠀⠀', 135)]

##currency symbol

In [28]:
currency_summary = adv.extract_currency(data['caption'])

In [29]:
currency_summary['top_currency_symbols']

[('£', 3791), ('$', 308), ('€', 23)]

##Frame dataset

In [30]:
# word_summary = adv.extract_words(data['caption'],entire_words_only=False)

In [31]:
extracted_tweets =  (data.assign(hashtags=hashtag_summary['hashtags'],
         hashcounts=hashtag_summary['hashtag_counts'],
         mentions=mention_summary['mentions'],
         mention_count=mention_summary['mention_counts'],
         emoji=emoji_summary['emoji'],
          emoji_text=emoji_summary['emoji_text'],
         emoji_count=emoji_summary['emoji_counts']         
         ))

extracted_tweets.head(5)

,caption,profile_name,biography,likes,followers,TextBlob_Analysis,hashtags,hashcounts,mentions,mention_count,emoji,emoji_text,emoji_count
10,𝗗𝗶𝗱 𝘆𝗼𝘂 𝗸𝗻𝗼𝘄 𝘁𝗵𝗮𝘁 𝘆𝗼𝘂 𝗰𝗮𝗻 𝘁𝗲𝘀𝘁 𝗳𝗼𝗿 𝗖𝗼𝘃𝗶𝗱 𝘄𝗶𝘁𝗵 ...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,14.0,1093.0,Positive,"[#londontesting, #chelseabridgeclinic, #london...",8,[],0,"[⏱, 📄, 🧑🏻‍⚕️, 👨‍👩‍👧‍👦, 👫, 🧑‍💻, 🤍, 📌, 🦠, 🤝]","[stopwatch, page facing up, health worker: lig...",10
11,𝗧𝗛𝗔𝗡𝗞 𝗬𝗢𝗨 👏\n.\nWe are very grateful for all t...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,12.0,1093.0,Positive,"[#thankyou, #thankful, #feedback, #chlseabridg...",9,[],0,"[👏, 🧡]","[clapping hands, orange heart]",2
12,"Hand dived Isle of Jura scallop, smoked roe, l...",Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,100.0,4202.0,Neutral,"[#edinburghfoodies, #edinburghrestaurants, #nu...",25,[],0,[],[],0
13,Jerusalem Artichoke prepared as your first des...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,81.0,4202.0,Positive,"[#michelin, #michelinstar, #michelinuk, #edinb...",28,[],0,[],[],0
14,Main course from our seasonal 7 course menu: B...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,146.0,4202.0,Positive,"[#michelin, #michelinstar, #michelinuk, #edinb...",28,"[@castlegamescotland, @wellocksfood]",2,[],[],0


In [32]:
le = preprocessing.LabelEncoder()
le.fit(extracted_tweets["TextBlob_Analysis"])
list(le.classes_)
account_num=le.transform(extracted_tweets["TextBlob_Analysis"])
extracted_tweets["TextBlob_Analysis"]=account_num

In [33]:
type(extracted_tweets)

pandas.core.frame.DataFrame

Let's say 10 people, each with 100 followers posted using the hashtag #basketball in their instagram.

Therefore those post are expected to potentially reach 10 x 100 = 1,000 people (not really that much because of follower overlap, and not everyone sees all post, but we will use this simplified assumption for now).
Let's also assume that only one person used the hashtag #NFL in these post. And let's also assume that this person has 20,000 followers. Then which hashtag reached more users?
Obviously #NFL would reach more because, although only one person posted it, that one person had more followers than the ten others who posted #basketball combined.



https://www.datacamp.com/community/tutorials/absolute-weighted-word-frequency

In [34]:
word_freq_hash = adv.word_frequency(extracted_tweets['hashtags'].str.join(' '), extracted_tweets['followers'].fillna(0))
word_freq_hash.head()
# type(word_freq_hash.word)

,word,abs_freq,wtd_freq,rel_value
0,#london,4737,98446898.0,20783.0
1,#plasticsurgery,3423,78919745.0,23056.0
2,#peggyporschen,121,69934975.0,577975.0
3,#cosmeticsurgery,2753,58649608.0,21304.0
4,#skincare,1709,52026462.0,30443.0


In [35]:
d=pd.Series(word_freq_hash.rel_value.values,index=word_freq_hash.word.values ).to_dict()
print(d.get('#chelseabridgeclinic'))
print(d.get('#london'))
print(d.get('#test'))

1093.0
20783.0
1093.0


In [36]:
def countHashtags(tags):
  if len(tags)==0:
    return 0
  sum = 0
  for i in tags:
    sum += (d.get(i))
  return sum/len(tags)

In [37]:
extracted_tweets['hashtags'][10]

['#londontesting',
 '#chelseabridgeclinic',
 '#london',
 '#test',
 '#antigentest',
 '#lateralflowtest',
 '#testforcovid',
 '#covidtesting']

In [38]:
hashtag_average = extracted_tweets['hashtags'].apply(countHashtags)

In [39]:
extracted_tweets['hashtag_popularity'] = hashtag_average

# extracted_tweets.drop(columns=extracted_tweets.columns[0], axis=1, inplace=True)

In [40]:
extracted_tweets.shape

(73232, 14)

In [41]:
# extracted_tweets['caption'][10]

Since the dataset is huge i have taken only 1000 rows to train the model

In [42]:
extracted_tweetss=extracted_tweets.iloc[:1000]

In [43]:
extracted_tweetss

,caption,profile_name,biography,likes,followers,TextBlob_Analysis,hashtags,hashcounts,mentions,mention_count,emoji,emoji_text,emoji_count,hashtag_popularity
10,𝗗𝗶𝗱 𝘆𝗼𝘂 𝗸𝗻𝗼𝘄 𝘁𝗵𝗮𝘁 𝘆𝗼𝘂 𝗰𝗮𝗻 𝘁𝗲𝘀𝘁 𝗳𝗼𝗿 𝗖𝗼𝘃𝗶𝗱 𝘄𝗶𝘁𝗵 ...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,14.0,1093.0,2,"[#londontesting, #chelseabridgeclinic, #london...",8,[],0,"[⏱, 📄, 🧑🏻‍⚕️, 👨‍👩‍👧‍👦, 👫, 🧑‍💻, 🤍, 📌, 🦠, 🤝]","[stopwatch, page facing up, health worker: lig...",10,3565.000000
11,𝗧𝗛𝗔𝗡𝗞 𝗬𝗢𝗨 👏\n.\nWe are very grateful for all t...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,12.0,1093.0,2,"[#thankyou, #thankful, #feedback, #chlseabridg...",9,[],0,"[👏, 🧡]","[clapping hands, orange heart]",2,6669.111111
12,"Hand dived Isle of Jura scallop, smoked roe, l...",Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,100.0,4202.0,1,"[#edinburghfoodies, #edinburghrestaurants, #nu...",25,[],0,[],[],0,21105.000000
13,Jerusalem Artichoke prepared as your first des...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,81.0,4202.0,2,"[#michelin, #michelinstar, #michelinuk, #edinb...",28,[],0,[],[],0,20552.714286
14,Main course from our seasonal 7 course menu: B...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,146.0,4202.0,2,"[#michelin, #michelinstar, #michelinuk, #edinb...",28,"[@castlegamescotland, @wellocksfood]",2,[],[],0,20552.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,Celebrate your #bankholiday in style this week...,Tsaretta Spice,"A vibrant, fine dining Indian restaurant exper...",12.0,1151.0,2,"[#bankholiday, #alfresco, #indiandining, #live...",10,[],0,[],[],0,8189.300000
1058,😎you might not be able to fly abroad this summ...,Tsaretta Spice,"A vibrant, fine dining Indian restaurant exper...",14.0,1151.0,2,"[#twickenham, #indiantapas, #gin, #vodka, #teq...",30,[],0,"[😎, 🥳, 🙌🏻, 💯, 🧉, 🍸, ☀️, 🍹]","[smiling face with sunglasses, partying face, ...",8,6886.866667
1059,🥳🧉🌶🛼We are open for delivery and collection! J...,Tsaretta Spice,"A vibrant, fine dining Indian restaurant exper...",28.0,1151.0,2,"[#indianfood, #indiancuisine, #indiantakeaway,...",29,[],0,"[🥳, 🧉, 🌶, 🛼]","[partying face, mate, hot pepper, roller skate]",4,6230.965517
1060,Do you fancy a little spice tonight? \nWhy not...,Tsaretta Spice,"A vibrant, fine dining Indian restaurant exper...",20.0,1151.0,0,"[#tapastuesday, #lovinglife, #livethelittlethi...",22,[],0,"[🌶, 🥳, ❤️, 😍, 🙌🏻]","[hot pepper, partying face, red heart, smiling...",5,9163.590909


##caption Bow

In [44]:
bow_vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 250)

bow_words = bow_vectorizer.fit_transform(extracted_tweetss['caption'])
bow_words.shape

(1000, 250)

In [45]:
bow_clean = bow_words.toarray()

vocab = bow_vectorizer.get_feature_names()
# print(vocab)


new_vocab = []
for word in vocab:
    if len(word)>5:
        new_vocab.append(word)
print (' '.join(new_vocab))

aesthetics always artdeco artdecohotel available balmoralmoments beautiful beauty before bishyroad chefsofinstagram christmas clinic cocktails coffee delicious delizioso dermal drjonathanroos eathealthy edinburgh edinburghcity edinburghfoodies edinburghlife edinburghrestaurants edinburghsnapshots eeeeeats everyone experience facerestorationlondon facial festive filler fillers finedining foodblogger foodgram foodie foodlovers foodphotography foodporn foodstagram foreveredinburgh health healthylifestyle homemadefood hotellife igersedinburgh independent instadaily instafood instagood instagram italiancuisine italianfood italianstyle krispykreme lecochonaveugle little london looking lovely mayfair meltonsrestaurant michelin michelinguide michelinstar michelinuk natural numberone numnum patient perfect photooftheday picoftheday plasticsurgery products restaurant restaurantsofinsta results roccofortefriends roccofortehotels scotland scotlandisnow season skincare something special street surg

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [46]:
words_clean = pd.DataFrame(data=bow_clean, columns=vocab)
words_clean.head()

,0203,2022,about,aesthetics,after,all,also,always,an,and,...,will,with,work,would,year,york,yorkfood,yorkshire,you,your
0,0,0,0,0,0,0,0,0,1,3,...,0,1,0,0,0,0,0,0,10,4
1,0,0,0,0,0,4,0,0,0,3,...,0,1,0,0,0,0,0,0,5,6
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [47]:
j = words_clean.columns.get_level_values(0).isin(new_vocab)
words_clean = words_clean.loc[:,j]
words_clean.head()

,aesthetics,always,artdeco,artdecohotel,available,balmoralmoments,beautiful,beauty,before,bishyroad,...,travel,traveller,treatment,twickenham,visitedinburgh,visitscotland,website,weekend,yorkfood,yorkshire
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,1,1,0,0,1,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,1,1,0,0,1,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,1,1,0,0,1,1,0,0,0,0


##Hashtag

In [48]:
from functools import reduce

def lower(text: str) -> str:
    return text




PREPROCESSING_PIPELINE = [lower]




# Anchor method

def text_prepare(text: str,filter_methods = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """

    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE

    return reduce(lambda txt, f: f(txt), filter_methods, text)


L=['hashtags','emoji_text','mentions']

for i in L:
  extracted_tweetss[i] = extracted_tweetss[i].apply(lambda txt: text_prepare(str(txt)[1:-1]))

extracted_tweetss

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,caption,profile_name,biography,likes,followers,TextBlob_Analysis,hashtags,hashcounts,mentions,mention_count,emoji,emoji_text,emoji_count,hashtag_popularity
10,𝗗𝗶𝗱 𝘆𝗼𝘂 𝗸𝗻𝗼𝘄 𝘁𝗵𝗮𝘁 𝘆𝗼𝘂 𝗰𝗮𝗻 𝘁𝗲𝘀𝘁 𝗳𝗼𝗿 𝗖𝗼𝘃𝗶𝗱 𝘄𝗶𝘁𝗵 ...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,14.0,1093.0,2,"'#londontesting', '#chelseabridgeclinic', '#lo...",8,,0,"[⏱, 📄, 🧑🏻‍⚕️, 👨‍👩‍👧‍👦, 👫, 🧑‍💻, 🤍, 📌, 🦠, 🤝]","'stopwatch', 'page facing up', 'health worker:...",10,3565.000000
11,𝗧𝗛𝗔𝗡𝗞 𝗬𝗢𝗨 👏\n.\nWe are very grateful for all t...,Chelsea Bridge Clinic,General Health | Orthopaedic Care | Advanced A...,12.0,1093.0,2,"'#thankyou', '#thankful', '#feedback', '#chlse...",9,,0,"[👏, 🧡]","'clapping hands', 'orange heart'",2,6669.111111
12,"Hand dived Isle of Jura scallop, smoked roe, l...",Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,100.0,4202.0,1,"'#edinburghfoodies', '#edinburghrestaurants', ...",25,,0,[],,0,21105.000000
13,Jerusalem Artichoke prepared as your first des...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,81.0,4202.0,2,"'#michelin', '#michelinstar', '#michelinuk', '...",28,,0,[],,0,20552.714286
14,Main course from our seasonal 7 course menu: B...,Number One,A 4 AA Rosette restaurant @thebalmoral #Edinbu...,146.0,4202.0,2,"'#michelin', '#michelinstar', '#michelinuk', '...",28,"'@castlegamescotland', '@wellocksfood'",2,[],,0,20552.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,Celebrate your #bankholiday in style this week...,Tsaretta Spice,"A vibrant, fine dining Indian restaurant exper...",12.0,1151.0,2,"'#bankholiday', '#alfresco', '#indiandining', ...",10,,0,[],,0,8189.300000
1058,😎you might not be able to fly abroad this summ...,Tsaretta Spice,"A vibrant, fine dining Indian restaurant exper...",14.0,1151.0,2,"'#twickenham', '#indiantapas', '#gin', '#vodka...",30,,0,"[😎, 🥳, 🙌🏻, 💯, 🧉, 🍸, ☀️, 🍹]","'smiling face with sunglasses', 'partying face...",8,6886.866667
1059,🥳🧉🌶🛼We are open for delivery and collection! J...,Tsaretta Spice,"A vibrant, fine dining Indian restaurant exper...",28.0,1151.0,2,"'#indianfood', '#indiancuisine', '#indiantakea...",29,,0,"[🥳, 🧉, 🌶, 🛼]","'partying face', 'mate', 'hot pepper', 'roller...",4,6230.965517
1060,Do you fancy a little spice tonight? \nWhy not...,Tsaretta Spice,"A vibrant, fine dining Indian restaurant exper...",20.0,1151.0,0,"'#tapastuesday', '#lovinglife', '#livethelittl...",22,,0,"[🌶, 🥳, ❤️, 😍, 🙌🏻]","'hot pepper', 'partying face', 'red heart', 's...",5,9163.590909


#Hashtag BOW

In [49]:
bow_vectorizer_hashtag = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 250)
# str(test_list)[1:-1]
bow_hashtag = bow_vectorizer_hashtag.fit_transform(extracted_tweetss['hashtags'])
bow_hashtag.shape

(1000, 250)

In [50]:
bow_hastag_clean = bow_hashtag.toarray()

vocab2 = bow_vectorizer_hashtag.get_feature_names()
print(vocab2)


new_vocab2 = []
for word in vocab2:
    if len(word)>5:
        new_vocab2.append(word)
print (' '.join(new_vocab2))

['abdominoplastia', 'abdominoplasty', 'abdominoplastybeforeandafter', 'abdominoplastypostop', 'abdominoplastyrecovery', 'abdominoplastyresults', 'acne', 'aesthetic', 'aesthetics', 'aestheticsurgery', 'antiaging', 'antiwrinkle', 'artdeco', 'artdecohotel', 'autumn', 'balmoralmoments', 'beach', 'beachvolley', 'beachvolleyball', 'beauty', 'becomingabetterme', 'beforeandafter', 'bestfoodworld', 'bestplasticsurgeon', 'beyourbestself', 'bishyroad', 'blepharoplasty', 'bodybydoctorv', 'bodycontouring', 'boobjob', 'botaniqualondon', 'breastaugmentation', 'breastenlargement', 'breasťaugmentation', 'brownhartgardens', 'cairngormcoffee', 'cairngorms', 'carboxytherapy', 'chanahchalska', 'chef', 'cheflife', 'chefs', 'chefsofinstagram', 'christmas', 'christmasseason', 'churchstreet', 'churchstreettwickenham', 'cityoflondon', 'cocktails', 'coffee', 'coffeeroaster', 'cooking', 'cosmeticsurgeon', 'cosmeticsurgery', 'cosmeticsurgeryclinic', 'cranleyclinic', 'curry', 'delicious', 'deliciousfood', 'delizios

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [51]:
clean_hashtag = pd.DataFrame(data=bow_hastag_clean, columns=vocab2)
clean_hashtag.head()

,abdominoplastia,abdominoplasty,abdominoplastybeforeandafter,abdominoplastypostop,abdominoplastyrecovery,abdominoplastyresults,acne,aesthetic,aesthetics,aestheticsurgery,...,wloskiesmaki,womeninhealthcare,workout,włoskiesmaki,york,yorkfood,yorkfoodanddrink,yorkrestaurants,yorkshire,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
j = clean_hashtag.columns.get_level_values(0).isin(new_vocab2)
clean_hashtag = clean_hashtag.loc[:,j]
clean_hashtag.head()

,abdominoplastia,abdominoplasty,abdominoplastybeforeandafter,abdominoplastypostop,abdominoplastyrecovery,abdominoplastyresults,aesthetic,aesthetics,aestheticsurgery,antiaging,...,whitton,wloskakuchnia,wloskiesmaki,womeninhealthcare,workout,włoskiesmaki,yorkfood,yorkfoodanddrink,yorkrestaurants,yorkshire
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##emoji's BOW

In [53]:
bow_vectorizer_emoji = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 250)
# str(test_list)[1:-1]
bow_emoji = bow_vectorizer_emoji.fit_transform(extracted_tweetss['emoji_text'])
bow_emoji.shape

(1000, 250)

In [54]:
bow_emoji_text_clean = bow_emoji.toarray()

vocab3 = bow_vectorizer_emoji.get_feature_names()
print(vocab3)


new_vocab3 = []
for word in vocab3:
    if len(word)>1:
        new_vocab.append(word)
print (' '.join(new_vocab3))

['africa', 'alarm', 'american', 'amulet', 'and', 'apple', 'arrow', 'away', 'backhand', 'bacon', 'ball', 'balloon', 'bar', 'beer', 'beverage', 'biceps', 'black', 'blowing', 'bottle', 'brain', 'brown', 'bullseye', 'button', 'calendar', 'camera', 'car', 'check', 'cheese', 'chick', 'chocolate', 'christmas', 'clapping', 'claus', 'clinking', 'clock', 'cocktail', 'collision', 'cookie', 'cork', 'couple', 'cream', 'crown', 'curving', 'cut', 'cyclone', 'dancing', 'dark', 'dashing', 'delivery', 'desert', 'dizzy', 'dog', 'doughnut', 'dove', 'down', 'drink', 'drooling', 'europe', 'evergreen', 'evil', 'exploding', 'eyes', 'face', 'fallen', 'fear', 'fire', 'flag', 'flash', 'flexed', 'folded', 'food', 'football', 'fork', 'french', 'fried', 'fries', 'getting', 'ghost', 'gift', 'glass', 'glasses', 'globe', 'glowing', 'green', 'grinning', 'growing', 'guitar', 'hamburger', 'hand', 'hands', 'hatching', 'head', 'health', 'heart', 'hearts', 'herb', 'hibiscus', 'high', 'horns', 'hospital', 'hot', 'hugging', '

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [55]:
clean_emoji_text = pd.DataFrame(data=bow_emoji_text_clean, columns=vocab3)
clean_emoji_text.head()

,africa,alarm,american,amulet,and,apple,arrow,away,backhand,bacon,...,wedge,white,wine,winking,with,without,woman,worker,wrapped,yellow
0,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,2,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
j3 = clean_emoji_text.columns.get_level_values(0).isin(new_vocab3)
clean_emoji_text = clean_emoji_text.loc[:,j3]
clean_emoji_text.head()

""
0
1
2
3
4


##mentions Bow

In [57]:
bow_vectorizer_mention = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 250)
# str(test_list)[1:-1]
bow_mention = bow_vectorizer_mention.fit_transform(extracted_tweetss['mentions'])
bow_mention.shape

(1000, 250)

In [58]:
bow_mention_clean = bow_mention.toarray()

vocab4 = bow_vectorizer_mention.get_feature_names()
print(vocab4)


new_vocab4 = []
for word in vocab4:
    if len(word)>2:
        new_vocab4.append(word)
print (' '.join(new_vocab4))

['2tall', '_mark_donald', 'abigailcp_455', 'aesthetic_medmag', 'aestheticclinicaltraining', 'aestheticmed_live', 'aestheticscourse', 'aestheticsjournaluk', 'aestheticsourceofficial', 'allergan_medical_institute', 'allerganaesthetics', 'alliancewine', 'arranbusk', 'aspinaloflondon', 'augustinusbader', 'babyfacedbaker', 'backroombar_london', 'barismercan', 'beach_volley_salitre_', 'beautifulpints', 'becsgentry', 'bendbeauty', 'bengals', 'bennettshomeprojects', 'blackcowvodka', 'blackpinecoffee', 'body_trade', 'britishthyroidfoundation', 'buddshirtmakers', 'budweiser', 'cave_du_cochon', 'caviarsturia', 'ccrlondon', 'cg_mediawork', 'charlesheidsieckchampagne', 'charmaineschow', 'chateaulepuy', 'cheekyyogaclub', 'chilternfirehouse', 'chivasregaluk', 'chorizovonsaucisse', 'cliniseptplus', 'cobiesmulders', 'cranleyclinic', 'cucinandomelagodo', 'dailyexpress', 'dailymail', 'dannykp_design', 'deepdishbeach', 'deepdishtour', 'dermaluxled', 'dermaroller_dermaspark', 'devoneelcompany', 'difference

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [59]:
clean_mention = pd.DataFrame(data=bow_mention_clean, columns=vocab4)
clean_mention.head()

,2tall,_mark_donald,abigailcp_455,aesthetic_medmag,aestheticclinicaltraining,aestheticmed_live,aestheticscourse,aestheticsjournaluk,aestheticsourceofficial,allergan_medical_institute,...,weare_ngx,wearemasuku,weetabixofficial,wellocksfood,wheresthatjoebuck,wildharbour,wilsonvolleyball,wiltshire_truffles,wsetglobal,yorkchristmastrees
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [60]:
j = clean_mention.columns.get_level_values(0).isin(new_vocab4)
clean_mention = clean_mention.loc[:,j]
clean_mention.head()

,2tall,_mark_donald,abigailcp_455,aesthetic_medmag,aestheticclinicaltraining,aestheticmed_live,aestheticscourse,aestheticsjournaluk,aestheticsourceofficial,allergan_medical_institute,...,weare_ngx,wearemasuku,weetabixofficial,wellocksfood,wheresthatjoebuck,wildharbour,wilsonvolleyball,wiltshire_truffles,wsetglobal,yorkchristmastrees
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [82]:
dataset1 = pd.DataFrame(np.hstack((words_clean,clean_hashtag,clean_mention,extracted_tweetss,clean_emoji_text)))

In [83]:
type(dataset1)

pandas.core.frame.DataFrame

In [95]:
dataset1[581]          

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: 581, Length: 1000, dtype: object

In [85]:
dataset1.columns                                      

RangeIndex(start=0, stop=598, step=1)

In [89]:
dataset1

,0,1,2,3,4,5,6,7,8,9,...,588,589,590,591,592,593,594,595,596,597
0,0,0,0,0,0,0,0,0,0,0,...,1093.0,2,"'#londontesting', '#chelseabridgeclinic', '#lo...",8,,0,"[⏱, 📄, 🧑🏻‍⚕️, 👨‍👩‍👧‍👦, 👫, 🧑‍💻, 🤍, 📌, 🦠, 🤝]","'stopwatch', 'page facing up', 'health worker:...",10,3565.0
1,0,0,0,0,0,0,0,0,0,0,...,1093.0,2,"'#thankyou', '#thankful', '#feedback', '#chlse...",9,,0,"[👏, 🧡]","'clapping hands', 'orange heart'",2,6669.111111
2,0,0,0,0,0,1,0,0,0,0,...,4202.0,1,"'#edinburghfoodies', '#edinburghrestaurants', ...",25,,0,[],,0,21105.0
3,0,0,0,0,0,1,0,0,0,0,...,4202.0,2,"'#michelin', '#michelinstar', '#michelinuk', '...",28,,0,[],,0,20552.714286
4,0,0,0,0,0,1,0,0,0,0,...,4202.0,2,"'#michelin', '#michelinstar', '#michelinuk', '...",28,"'@castlegamescotland', '@wellocksfood'",2,[],,0,20552.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,1151.0,2,"'#bankholiday', '#alfresco', '#indiandining', ...",10,,0,[],,0,8189.3
996,0,0,0,0,0,0,0,0,0,0,...,1151.0,2,"'#twickenham', '#indiantapas', '#gin', '#vodka...",30,,0,"[😎, 🥳, 🙌🏻, 💯, 🧉, 🍸, ☀️, 🍹]","'smiling face with sunglasses', 'partying face...",8,6886.866667
997,0,0,0,0,1,0,0,0,0,0,...,1151.0,2,"'#indianfood', '#indiancuisine', '#indiantakea...",29,,0,"[🥳, 🧉, 🌶, 🛼]","'partying face', 'mate', 'hot pepper', 'roller...",4,6230.965517
998,0,0,0,0,0,0,0,0,0,0,...,1151.0,0,"'#tapastuesday', '#lovinglife', '#livethelittl...",22,,0,"[🌶, 🥳, ❤️, 😍, 🙌🏻]","'hot pepper', 'partying face', 'red heart', 's...",5,9163.590909


In [97]:
dropped = [584,585,586,590,592,594,595]
data = dataset1.drop(dropped,axis=1)
data


# caption,profile_name,biography,hashtags,mentions,emoji,emoji_text

,0,1,2,3,4,5,6,7,8,9,...,581,582,583,587,588,589,591,593,596,597
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,14.0,1093.0,2,8,0,10,3565.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,12.0,1093.0,2,9,0,2,6669.111111
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,100.0,4202.0,1,25,0,0,21105.0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,81.0,4202.0,2,28,0,0,20552.714286
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,146.0,4202.0,2,28,2,0,20552.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,12.0,1151.0,2,10,0,0,8189.3
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,14.0,1151.0,2,30,0,8,6886.866667
997,0,0,0,0,1,0,0,0,0,0,...,0,0,0,28.0,1151.0,2,29,0,4,6230.965517
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,20.0,1151.0,0,22,0,5,9163.590909


In [99]:
data

,0,1,2,3,4,5,6,7,8,9,...,581,582,583,587,588,589,591,593,596,597
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,14.0,1093.0,2,8,0,10,3565.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,12.0,1093.0,2,9,0,2,6669.111111
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,100.0,4202.0,1,25,0,0,21105.0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,81.0,4202.0,2,28,0,0,20552.714286
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,146.0,4202.0,2,28,2,0,20552.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,12.0,1151.0,2,10,0,0,8189.3
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,14.0,1151.0,2,30,0,8,6886.866667
997,0,0,0,0,1,0,0,0,0,0,...,0,0,0,28.0,1151.0,2,29,0,4,6230.965517
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,20.0,1151.0,0,22,0,5,9163.590909


##TF-IDF

In [66]:
tfidf_vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 250)
tfidf = tfidf_vectorizer.fit_transform(extracted_tweetss['mentions'])
tfidf.shape

(1000, 250)

#word2vec

#doc2vec

In [67]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


# parameters = {
#             'max_depth':[3], 
#             'learning_rate':[0.48], 
#             'n_estimators':[361]
#          }
# xg = GridSearchCV(xgb.XGBRegressor(), parameters, cv = 5, scoring = 'neg_mean_squared_error', verbose=100)

In [100]:
y = np.ravel(data[587])

In [101]:
dataset1 = data.drop(587, 1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [70]:
# xg.fit(dataset1, y)
# xg.best_score_

In [71]:
# from sklearn.model_selection import cross_val_score
# # from sklearn.cross_validation import train_test_split

# model = xgb.XGBRegressor()
# scores = cross_val_score(model, dataset1, y, cv=5, scoring = 'neg_mean_squared_error')
# np.mean(scores)

In [102]:
x_train, x_test, y_train, y_test = train_test_split(dataset1, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor()
model.fit(x_train, y_train)
model.score(x_test,y_test)

testing = model.predict(x_test)

TypeError: ignored

In [103]:
rmse = np.sqrt(mean_squared_error(y_test, testing))
print("RMSE: %f" % (rmse))

R2=r2_score(y_test,testing)
print("R2 Score: %f" % (R2))
print()

NameError: ignored